---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one(filename = 'email_network.txt', header = 0):
    
    ## Import the text file
    #graph_df = pd.read_csv(filename, delimiter='\t', header=header)
    #graph_df.columns = ['Sender', 'Recipient', 'time']
    edges = []
    with open(filename, 'r') as f:
        for line in f.readlines()[1:]:
            edges.append( (line.split('\t')[0], line.split('\t')[1], {'time': line.split('\t')[2].replace('\n', '')}) )
    
    ## Initialize a multi-digraph and grow it by adding edges
    G = nx.MultiDiGraph()
    #edges = [(row['Sender'], row['Recipient'], dict(time = row['time'])) 
    #            for i, row in graph_df.iterrows()]
    G.add_edges_from(edges)
    
    return G

G = answer_one()
print(G.is_directed())
print(G.is_multigraph())
print(G.edges(data = True)[:4]) ## check out the first 5 edges

True
True
[('1', '2', {'time': '1262454010'}), ('1', '2', {'time': '1262604260'}), ('1', '2', {'time': '1262645364'}), ('1', '2', {'time': '1262734830'})]


### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [3]:
def answer_two(G = G):
    
    n_employees = len(G.nodes())
    n_emails = len(G.edges())
    
    return (n_employees, n_emails)

answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [4]:
def answer_three(G = G):
        
    part1 = nx.is_strongly_connected(G)
    
    part2 = nx.is_weakly_connected(G)
    
    return (part1, part2)

answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [5]:
def answer_four(G = G):
        
    components = nx.weakly_connected_components(G)
    lens = [len(comp) for comp in components]
    max_len = max(lens)
    
    return max_len

answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [6]:
def answer_five(G = G):
        
    components = nx.strongly_connected_components(G)
    lens = [len(comp) for comp in components]
    max_len = max(lens)
    
    return max_len

answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [7]:
def answer_six(G = G):
        
    subgraphs = nx.strongly_connected_component_subgraphs(G)
    for G_sc in subgraphs:
        if len(G_sc.nodes()) == answer_five():
            return G_sc
        else:
            pass

G_sc = answer_six()
G_sc

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [8]:
def answer_seven(G_sc = G_sc):
        
    avg_dist = nx.average_shortest_path_length(G_sc)
    
    return avg_dist

answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [9]:
def answer_eight(G_sc = G_sc):
    
    diameter = nx.diameter(G_sc)
    
    return diameter

answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [10]:
def answer_nine(G_sc = G_sc):
    
    periphery = set(nx.periphery(G_sc))
    
    return periphery

answer_nine()

{'129', '134', '97'}

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [11]:
def answer_ten(G_sc = G_sc):
    
    center = set(nx.center(G_sc))
    
    return center

answer_ten()

{'38'}

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [12]:
def answer_eleven(G_sc = G_sc):
    
    periphery_node_info = {}
    for node in answer_nine():
        diameter_path_count = 0
        connection_count = 0
        for other_node in [node2 for node2 in G_sc.nodes() if node2 != node]:
            dist = nx.shortest_path_length(G_sc, node, other_node)
            if dist == answer_eight():
                diameter_path_count+=1
            elif dist == 1:
                connection_count+=1
            else:
                pass
        periphery_node_info[node] = {'connection_count': connection_count,
                                     'diameter_path_count': diameter_path_count}
    
    max_diameter_path_count = 0
    node_name = ''
    for node in periphery_node_info:
        if periphery_node_info[node]['diameter_path_count'] > max_diameter_path_count:
            max_diameter_path_count = periphery_node_info[node]['diameter_path_count']
            node_name = node
    n_connected_nodes = periphery_node_info[node_name]['diameter_path_count'] #connection_count
    
    return (node_name, n_connected_nodes)

answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [18]:
def answer_twelve(G_sc = G_sc):
    
    center_nodes = answer_ten()
    center_node = list(center_nodes)[0]
    
    periphery_node_tuple = answer_eleven()
    periphery_node = periphery_node_tuple[0]
    
    min_nodes = nx.minimum_node_cut(G_sc, periphery_node, center_node)
    
    return len(min_nodes)

answer_twelve()

1

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [15]:
def answer_thirteen(G_sc = G_sc):
    
    edges = list(set(G_sc.edges()))
    
    G_un = nx.Graph()
    G_un.add_edges_from(edges)
    
    return G_un

G_un = answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [16]:
def answer_fourteen(G_un = G_un):
    
    trans = nx.transitivity(G_un)
    avg_cluster = nx.average_clustering(G_un)
    
    return (trans,avg_cluster)

answer_fourteen()

(0.570111160700385, 0.697527243723142)